In [ ]:
from tqdm.notebook import tqdm
import os
import requests
import json
import pandas as pd

version_to_run = "v1.0.0"
    
###################################################################################################################
tasks_path = "data/filtered_run_path"
#tasks_list = os.listdir(os.path.join(tasks_path, version_to_run))

def get_json_from_url(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for 4xx or 5xx status codes
        json_data = response.json()
        return json_data
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None
        
overwrite = False

In [ ]:
df = pd.read_csv('helm_lite.csv')


In [ ]:
df.columns

In [ ]:
df


In [ ]:
list(df.Model.unique())

In [ ]:
df.Groups.value_counts()

In [ ]:
df[["Groups", "Subject / Task"]].value_counts()

In [ ]:
tasks_list = list(df.Run)

In [ ]:
tasks_list

In [ ]:
import os, sys
wd = os.getcwd()

In [ ]:
template_url = f"https://storage.googleapis.com/crfm-helm-public/lite/benchmark_output/runs/{version_to_run}"
save_dir = f"{wd}/../../raw_results/helm_lite_{version_to_run}"

The following cell downloads all the raw data.

In [ ]:
for tasks in [tasks_list]:
    
    for task in tqdm(tasks):
        cur_save_dir = f"{save_dir}/{task}"
        os.makedirs(cur_save_dir, exist_ok=True)

        for file_type in [
                # "run_spec",
                # "stats",
                # "per_instance_stats",
                # "instances",
                "scenario_state",  # this file contains all the useful bits
                "display_predictions",
                # "display_requests",
                # "scenario",
        ]:
            save_path = f"{cur_save_dir}/{file_type}.json"
            if os.path.exists(save_path):
                with open(save_path) as f:
                    data = json.load(f)
                if data is None or overwrite:
                    download = True
                else: 
                    download = False
            else:
                download = True
                                
            #https://storage.googleapis.com/crfm-helm-public/benchmark_output/runs/v0.2.2/babi_qa:task=15,model=AlephAlpha_luminous-base/scenario_state.json
            
            if download:            
                cur_url = f"{template_url}/{task}/{file_type}.json"
                print("download ",  cur_url)
                json.dump(get_json_from_url(cur_url), open(save_path, "w"), indent=2)